In [1]:
import pandas as pd
import numpy as np
import string
import spacy
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import CountVectorizer

nlp = spacy.load('en_core_web_sm')

In [2]:
starfield_reviews_df = pd.read_csv('starfield_reviews.csv')

In [3]:
# Remove as linhas com valores nulos do DataFrame starfield_reviews_df
starfield_reviews_df.dropna(inplace=True)

# Após a remoção, você pode querer redefinir os índices do DataFrame
starfield_reviews_df.reset_index(drop=True, inplace=True)


In [5]:
starfield_reviews_df.head()

,review_id,text,target
0,147134704,Uninstalled until they fix and improve this ga...,0
1,147134681,badass,1
2,147134591,TL:DR Not worth the whole price. It's a basic ...,0
3,147134557,Only a few hours in- LOVE it. Yeah maybe a few...,1
4,147134157,Great overall game. Some systems are flawed or...,1


In [6]:
starfield_reviews_df['text'] = starfield_reviews_df['text'].apply(lambda x: [token for token in x if token not in string.punctuation and not token.isnumeric()])
starfield_reviews_df['text'] = starfield_reviews_df['text'] .apply(lambda x: ''.join(x))

In [7]:
starfield_reviews_df['text']  = starfield_reviews_df['text'].apply(lambda x: [token.lemma_.lower() for token in nlp(x) if (token.is_stop == False and len(token.text)>2)])
starfield_reviews_df['text']  = starfield_reviews_df['text'].apply(lambda x: ' '.join(x))

In [8]:

X_train, X_test, y_train, y_test = train_test_split(starfield_reviews_df['text'], starfield_reviews_df['target'], test_size=0.2, random_state=42, stratify=starfield_reviews_df['target'])

In [9]:
# Crie um pipeline com o CountVectorizer para representação BoW e o SVM como classificador
pipeline = Pipeline([
    ('vectorizer', CountVectorizer()),  # Representação Bag of Words
    ('classifier', SVC(kernel='linear'))  # Classificador SVM
])

# Treine o modelo
pipeline.fit(X_train, y_train)

# Faça previsões no conjunto de teste
y_pred = pipeline.predict(X_test)


In [10]:
# Avaliar o desempenho do modelo
accuracy = accuracy_score(y_test, y_pred)
print(f'Acurácia: {accuracy:.2f}')

# Exibir um relatório de classificação
report = classification_report(y_test, y_pred)
print('Relatório de Classificação:\n', report)

Acurácia: 0.89
Relatório de Classificação:
               precision    recall  f1-score   support

           0       0.82      0.67      0.74      1918
           1       0.91      0.96      0.93      6587

    accuracy                           0.89      8505
   macro avg       0.86      0.81      0.84      8505
weighted avg       0.89      0.89      0.89      8505

